In [ ]:
# 開発過程の補足

# この Jupyter Notebook ファイルは、開発過程を示すために作成されています。  
# セル単位でコードを実行しながら処理内容を確認する「検証用環境」として使用しています。  
# 実際の本番環境では VSCode 上でモジュールを作成し、AWS Lambda および Step Functions によりデータ処理を実行しています。

In [ ]:
# 開発履歴参照のため残しておく。実行は「yahoo_rakuten_price.py」。

# セル1: 共通ライブラリとクラス定義

import os
import yaml
import json
import requests
import pandas as pd
import time
from dotenv import load_dotenv
from datetime import datetime
import logging
from pathlib import Path
import csv

class YahooRakutenPriceSearch:
    """
    ヤフーショッピングと楽天のAPIを使用して商品の最安値情報を検索するクラス
    """
    
    def __init__(self, config_path=None):
        """
        YahooRakutenPriceSearchクラスの初期化
        
        Parameters:
            config_path (str, optional): 設定ファイルのパス
        """
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # 環境変数の読み込み
        load_dotenv(os.path.join(self.root_dir, '.env'))
        
        # ディレクトリパスの設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリが存在しない場合は作成
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        
        # 設定ファイルの読み込み
        self.config = self._load_config(config_path)
        
        # APIキーの設定（環境変数から取得）
        self.yahoo_client_id = os.getenv('YAHOO_CLIENT_ID')
        if not self.yahoo_client_id:
            raise ValueError("YAHOO_CLIENT_IDが環境変数に設定されていません")
            
        self.rakuten_application_id = os.getenv('RAKUTEN_APPLICATION_ID')
        if not self.rakuten_application_id:
            # 環境変数がない場合はデフォルト値を使用
            self.rakuten_application_id = "1015757852035447235"
            print(f"環境変数RAKUTEN_APPLICATION_IDが設定されていないため、デフォルト値を使用します: {self.rakuten_application_id}")
        
        # ログ設定
        self.setup_logging()
        
        # APIのベースURL
        self.yahoo_base_url = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
        self.rakuten_base_url = "https://app.rakuten.co.jp/services/api/IchibaItem/Search/20220601"
        
        # 出力ファイル名
        self.output_file = os.path.join(self.data_dir, self.config['price_search']['output']['output_file'])
    
    def _find_project_root(self):
        """プロジェクトのルートディレクトリを検出する"""
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            if (path / 'setup.py').exists() or (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent
    
    def _load_config(self, config_path=None):
        """設定ファイルを読み込む"""
        # デフォルトのパス
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            # YAMLファイルを読み込む
            with open(config_path, 'r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
            
            # デフォルト設定の追加
            if 'price_search' not in config:
                config['price_search'] = {}
            
            # 出力設定（なければデフォルト値を設定）
            if 'output' not in config['price_search']:
                config['price_search']['output'] = {
                    'csv_filename': 'yahoo_rakuten_price_output.csv',
                    'input_file': 'jan_list.csv'
                }
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            print(f"設定ファイルの読み込みエラー: {str(e)}")
            # エラーを上位に伝播させる
            raise
    
    def setup_logging(self):
        """ログ機能のセットアップ"""
        # すでに存在するハンドラを削除（重複を防ぐため）
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
        
        # ログファイルパスの設定
        log_filename = f'price_search_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
        log_file = os.path.join(self.log_dir, log_filename)
        
        # 基本設定
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            encoding='utf-8'
        )
        
        # コンソールにもログを出力
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)
        
        # ログファイルの場所を明示的に表示
        print(f"ログファイル出力先: {log_file}")
        logging.info(f"ログ機能の初期化が完了しました: {log_file}")
        
    def load_jan_codes(self, input_file=None):
        """
        CSVファイルからJANコードを読み込む
        
        Parameters:
            input_file (str, optional): 入力CSVファイルのパス
            
        Returns:
            list: JANコードのリスト
        """
        # 入力ファイルのパスを設定
        if input_file is None:
            input_file = os.path.join(self.data_dir, self.config['price_search']['output']['input_file'])
        
        jan_codes = []
        
        try:
            if not os.path.exists(input_file):
                raise FileNotFoundError(f"入力ファイルが見つかりません: {input_file}")
            
            # CSVファイルの読み込み
            with open(input_file, 'r', encoding='utf-8-sig') as f:
                reader = csv.reader(f)
                header = next(reader, None)  # ヘッダー行を読み飛ばす
                
                # JANコード列を探す
                jan_col = 0
                if header:
                    for i, col in enumerate(header):
                        if 'JAN' in col.upper() or 'コード' in col or 'CODE' in col.upper():
                            jan_col = i
                            break
                
                # 各行からJANコードを取得
                for row in reader:
                    if row and len(row) > jan_col and row[jan_col].strip():
                        jan_code = row[jan_col].strip()
                        # 数字のみで構成されているか確認
                        if jan_code.isdigit():
                            jan_codes.append(jan_code)
            
            logging.info(f"{len(jan_codes)}件のJANコードを読み込みました")
            print(f"{len(jan_codes)}件のJANコードを読み込みました: {input_file}")
            
            return jan_codes
            
        except Exception as e:
            logging.error(f"JANコード読み込みエラー: {str(e)}")
            print(f"JANコード読み込みエラー: {str(e)}")
            return []
    
    def save_to_csv(self, products, append=False):
        """
        商品情報をCSVファイルに保存
        
        Parameters:
            products (list): 商品情報のリスト
            append (bool): 追記モードか上書きモードか
        """
        if not products:
            logging.warning("保存する商品情報がありません")
            return
        
        # DataFrameに変換
        df = pd.DataFrame(products)
        
        # 保存モード
        mode = 'a' if append else 'w'
        header = not append or not os.path.exists(self.output_file)
        
        # CSVに保存
        df.to_csv(self.output_file, mode=mode, index=False, encoding='utf-8-sig', header=header)
        
        action = "追記" if append else "保存"
        logging.info(f"{len(products)}件の商品情報を{action}しました: {self.output_file}")
        print(f"{len(products)}件の商品情報を{action}しました: {self.output_file}")

In [ ]:
# セル2: ヤフーショッピングAPI実装部分

# yahoo_search_by_jan メソッドを修正
def yahoo_search_by_jan(self, jan_code, max_items=3):
    """
    ヤフーショッピングAPIでJANコードから商品を検索し、最安値順に上位の商品情報を取得する
    
    Parameters:
        jan_code (str): 検索するJANコード
        max_items (int): 取得する最大商品数（デフォルト: 3）
        
    Returns:
        list: 商品情報のリスト（価格、送料条件、商品URL）
    """
    # パラメータの設定
    params = {
        'appid': self.yahoo_client_id,
        'jan_code': jan_code,  # JANコードで検索
        'availability': 1,     # 在庫あり
        'condition': 'new',    # 新品
        'sort': '+price',      # 価格の安い順
        'results': max_items   # 取得する商品数
    }
    
    try:
        # APIリクエスト
        logging.info(f"Yahoo: JANコード {jan_code} の商品検索を開始")
        print(f"Yahoo: JANコード {jan_code} の商品検索中...")
        
        response = requests.get(self.yahoo_base_url, params=params)
        
        # HTTPステータスコードのチェック
        response.raise_for_status()
        
        # JSONに変換
        data = response.json()
        
        # 取得した商品情報を整形
        products = []
        for i, hit in enumerate(data.get('hits', []), 1):
            price = hit.get('price', 0)
            shipping_condition = hit.get('shipping', {}).get('name', '不明') if hit.get('shipping') else '不明'
            
            # 送料条件に応じて価格_条件込みを設定
            if shipping_condition == '送料無料':
                formatted_price = str(price)  # そのまま
            elif shipping_condition == '条件付き送料無料':
                formatted_price = f'〈{price}〉'  # 〈〉で囲む
            else:  # '送料別'や'設定なし'の場合
                formatted_price = f'【{price}】'  # 【】で囲む
            
            product = {
                'JAN': jan_code,
                'API': f'Yahoo{i}',
                '価格': price,
                '送料条件': shipping_condition,
                '価格_条件込み': formatted_price,
                '商品URL': hit.get('url', '')
            }
            products.append(product)
        
        logging.info(f"Yahoo: JANコード {jan_code} の商品検索結果: {len(products)}件")
        return products
        
    except requests.exceptions.RequestException as e:
        logging.error(f"Yahoo APIリクエストエラー: {str(e)}")
        return []

# クラスにメソッドを追加
YahooRakutenPriceSearch.yahoo_search_by_jan = yahoo_search_by_jan

In [ ]:
# セル3: 楽天API実装部分

# rakuten_search_by_jan メソッドを修正
def rakuten_search_by_jan(self, jan_code, max_items=3):
    """
    楽天APIでJANコードから商品を検索し、最安値順に上位の商品情報を取得する
    
    Parameters:
        jan_code (str): 検索するJANコード
        max_items (int): 取得する最大商品数（デフォルト: 3）
        
    Returns:
        list: 商品情報のリスト（価格、送料条件、商品URL）
    """
    # パラメータの設定
    params = {
        'applicationId': self.rakuten_application_id,
        'keyword': jan_code,            # JANコードをキーワードとして検索
        'hits': max_items,              # 最大取得件数
        'sort': '+itemPrice',           # 価格の安い順にソート
        'format': 'json',               # レスポンス形式
        'formatVersion': 2              # API形式バージョン
    }
    
    try:
        # APIリクエスト
        logging.info(f"Rakuten: JANコード {jan_code} の商品検索を開始")
        print(f"Rakuten: JANコード {jan_code} の商品検索中...")
        
        response = requests.get(self.rakuten_base_url, params=params)
        
        # HTTPステータスコードのチェック
        response.raise_for_status()
        
        # JSONに変換
        data = response.json()
        
        # 取得した商品情報を整形
        products = []
        
        # Itemsが存在するかチェック
        if 'Items' in data and data['Items']:
            for i, item in enumerate(data['Items'], 1):
                price = item.get('itemPrice', 0)
                shipping_condition = '送料込み' if item.get('postageFlag', 0) == 0 else '送料別'
                
                # 送料条件に応じて価格_条件込みを設定
                if shipping_condition == '送料込み':
                    formatted_price = str(price)  # そのまま
                else:  # '送料別'の場合
                    formatted_price = f'【{price}】'  # 【】で囲む
                
                # URLからアフィリエイト部分を削除
                original_url = item.get('itemUrl', '')
                clean_url = original_url.split('?')[0]  # '?'以降を削除
                
                # 商品情報の直接取得
                product = {
                    'JAN': jan_code,
                    'API': f'Rakuten{i}',
                    '価格': price,
                    '送料条件': shipping_condition,
                    '価格_条件込み': formatted_price,
                    '商品URL': clean_url
                }
                
                # 有効な情報が取得できた場合のみ追加
                if product['価格'] > 0 or product['商品URL']:
                    products.append(product)
            
            logging.info(f"Rakuten: JANコード {jan_code} の商品検索結果: {len(products)}件")
        else:
            logging.info(f"Rakuten: JANコード {jan_code} の商品情報が見つかりません")
        
        return products
        
    except requests.exceptions.RequestException as e:
        logging.error(f"Rakuten APIリクエストエラー: {str(e)}")
        return []

# クラスにメソッドを追加
YahooRakutenPriceSearch.rakuten_search_by_jan = rakuten_search_by_jan

In [ ]:
# セル4: JAN読み込みと交互検索の実装

# YahooRakutenPriceSearchクラスに交互検索機能を追加
def alternating_search(self, input_file=None, max_items=3):
    """
    JANコードをCSVから読み込み、ヤフーショッピングと楽天APIを交互に使用して検索
    ヤフーショッピングAPI: 2秒間隔
    楽天API: 1秒間隔
    
    Parameters:
        input_file (str, optional): 入力ファイルのパス
        max_items (int): 各JANコードごとに取得する最大商品数
        
    Returns:
        int: 検索した総商品数
    """
    # JANコードの読み込み
    jan_codes = self.load_jan_codes(input_file)
    
    if not jan_codes:
        logging.warning("処理対象のJANコードがありません")
        return 0
    
    # 初回は上書きモードで保存フラグを設定（空のレコードは作成しない）
    first_save = True
    
    # 総検索商品数
    total_items = 0
    
    # 前回のYahoo APIリクエスト時刻を記録
    last_yahoo_request = time.time()
    
    # JANコードごとに交互検索
    for i, jan_code in enumerate(jan_codes):
        print(f"\n===== JANコード {jan_code} の検索を開始 ({i+1}/{len(jan_codes)}) =====")
        
        # 1. Yahoo検索（最低2秒間隔を確保）
        current_time = time.time()
        time_since_last_yahoo = current_time - last_yahoo_request
        if time_since_last_yahoo < 2.0:
            wait_time = 2.0 - time_since_last_yahoo
            print(f"Yahoo APIの制限のため {wait_time:.2f}秒 待機します...")
            time.sleep(wait_time)
        
        # Yahoo検索実行
        yahoo_products = self.yahoo_search_by_jan(jan_code, max_items)
        last_yahoo_request = time.time()  # Yahoo API呼び出し時刻を更新
        
        if yahoo_products:
            # 初回は新規作成、2回目以降は追記
            self.save_to_csv(yahoo_products, append=not first_save)
            if first_save:
                first_save = False
            total_items += len(yahoo_products)
        
        # Yahoo API呼び出し後の安全な待機（1秒）
        time.sleep(1.0)
        
        # 2. Rakuten検索
        rakuten_products = self.rakuten_search_by_jan(jan_code, max_items)
        if rakuten_products:
            # 初回は新規作成、2回目以降は追記
            self.save_to_csv(rakuten_products, append=not first_save)
            if first_save:
                first_save = False
            total_items += len(rakuten_products)
        
        # Rakuten API呼び出し後の待機（1秒）
        if i < len(jan_codes) - 1:
            time.sleep(1.0)
    
    # 統計情報
    print(f"\n===== 検索完了 =====")
    print(f"処理したJANコード数: {len(jan_codes)}")
    print(f"取得した商品情報数: {total_items}")
    
    return total_items

# クラスにメソッドを追加
YahooRakutenPriceSearch.alternating_search = alternating_search

In [ ]:
# セル5: メイン処理（実行）

if __name__ == "__main__":
    try:
        # YahooRakutenPriceSearchのインスタンスを作成
        price_search = YahooRakutenPriceSearch()
        
        # 交互検索を実行
        start_time = time.time()
        total_items = price_search.alternating_search()
        end_time = time.time()
        
        # 実行時間の表示
        elapsed = end_time - start_time
        print(f"処理時間: {elapsed:.2f}秒")
        
        # 成功メッセージ
        if total_items > 0:
            print(f"✅ 処理が完了しました！合計{total_items}件の商品情報を取得しました")
            print(f"結果は {os.path.join(price_search.data_dir, 'yahoo_rakuten_price_output.csv')} に保存されました")
        else:
            print("❌ 商品情報を取得できませんでした")
        
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        import traceback
        traceback.print_exc()